## Языковое моделирование

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста. Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).

Подсказки:
- нужно будет добавить еще один тэг
- еще одна матрица не нужна, можно по строкам хранить биграмы, а по колонкам униграммы
- тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так).

In [1]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
from nltk.tokenize import sent_tokenize
from scipy.sparse import lil_matrix

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
news = open('/content/drive/MyDrive/ВШЭ/Магистратура/NLP/lenta.txt').read()

Напишем простую функцию для нормализации. Удалять пунктуацию и приводить к нижнему регистру, строго говоря не стоит, сгенерированный текст так будет не похож на настоящий. Но это немного упростит нам работу.

In [6]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [9]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [10]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [11]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, 2))
    trigrams_news.update(ngrammer(sentence, 3))

In [14]:
trigrams_news.most_common(10)

[('<start> <start> в', 3272),
 ('<start> <start> по', 2577),
 ('<start> <start> как', 1794),
 ('<start> <start> однако', 714),
 ('<start> <start> на', 684),
 ('<start> <start> об', 684),
 ('<start> об этом', 662),
 ('<start> <start> он', 653),
 ('<start> по словам', 630),
 ('<start> как сообщает', 611)]

### Генерация текста

In [15]:
matrix_news = lil_matrix((len(bigrams_news),
                        len(unigrams_news)))

id2word_news_1 = list(unigrams_news)
word2id_news_1 = {word:i for i, word in enumerate(id2word_news_1)}
id2word_news_2 = list(bigrams_news)
word2id_news_2 = {word:i for i, word in enumerate(id2word_news_2)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = ' '.join([word1, word2])
    unigram = word3
    matrix_news[word2id_news_2[bigram], word2id_news_1[unigram]] =  (trigrams_news[ngram]/bigrams_news[bigram])

In [18]:
matrix_news

<380345x71987 sparse matrix of type '<class 'numpy.float64'>'
	with 563892 stored elements in List of Lists format>

Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [31]:
def generate(matrix, id2word, word2id, n=100, start='<start> <start>'):
    text = []
    current_idx = word2id[start]
    current_bigram = start

    for i in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        # print(current_bigram.split(), chosen, id2word[chosen])
        text.append(id2word[chosen])
        current_bigram = f'{current_bigram.split()[1]} {id2word[chosen]}'


        if id2word[chosen] == '<end>':
            chosen = word2id[start]
            current_bigram = start
        current_idx = word2id[current_bigram]

    return ' '.join(text)

In [32]:
print(generate(matrix_news, id2word_news_1, word2id_news_2).replace('<end>', '\n'))

в департаменте правительственной информации соответствующее распоряжение было подписано еще 17 человек погибших за время фунционирования горячей линии сообщать об этих проблемах дает заявление министра иностранных дел панамы приветствовал джимми картера как главу американской делегации деликатно не обратив внимание на незнакомых людей выносящих мусор в контейнеры на припаркованные к домам подозрительные грузовики 
 касаясь дела о его отставке 
 тогда фсбэшники появились перед журналистами мадлен олбрайт отметила что политические проблемы 
 позвонки лежатодин к одному сохранность их идеальная 
 таким образом чтобы не допустить дальнейшее осуществление неправомерных валютных трансфертов 
 как сообщил интерфаксу сам арсаханов возглавляемая им международная организация готова подключиться


In [33]:
print(generate(matrix_news, id2word_news_1, word2id_news_2).replace('<end>', '\n'))

12 октября состоялось одновременное открытие двух интернет-центров в калининграде и 
 а после выборов специализированный ресурс полит ру проявляют повышенный интерес к которым уже истрачено около 22 миллиардов до 19,5 миллиардов кубометров газа 
 это крупнейшее из проводимых когда-либо исследований такого рода преступления не находятся в госпиталях в тяжелом состоянии передает reuters 
 от урагана пострадали также литва и чехия примут участие известныеэстрадные исполнители 
 представитель этой организации составляет более 104 миллионов рублей в месяц заявил он 
 я убил более ста мальчиков и растворил части их тел в кислоте 
 думаю это была самая крупная авария на гонках формула-1


In [34]:
print(generate(matrix_news, id2word_news_1, word2id_news_2).replace('<end>', '\n'))

это означает что американцы установили систему прослушивания телефонов в нашем стиле работы и на банковские военные и другие на дополнительную проверку направлено более 15 пассажиров следующих в аэропорт горячую еду в пятницу утверждение президента ингушетии руслана аушева среди этих людей немало глубоких стариков и женщин которые мешают искать бандитов и террористов в чечне 
 в этом году в минувший четверг в ходе следствия было установлено заказчиком книги является находящаяся в распоряжении трибунала команда судебно-медицинских экспертов из forrester research к 2003 году и зарегистрировано в установленном порядке оформить въездные документы в здание вошли заместитель главы администрации президента 
 в качестве легального


In [35]:
print(generate(matrix_news, id2word_news_1, word2id_news_2).replace('<end>', '\n'))

в интервью корреспонденту wall street journal europe сообщил о своем отношении к россии связанных с делом bank of new york post джексон поясняя свою точку зрения что продолжение разработок в области высоких технологий получат преимущество 
 мстислав ростропович выступит также отдельно с виолончелистами и с кем и кем иностранные миссии собираются выступать посредниками 
 горело трехэтажное здание старой постройки 
 предварительное расследование показало что при подписании союзного договора вместе и энергично подчеркнул глава государства запретил совмещать посты заместителя министра в ходе телефонного разговора с московской патриархией уже шестой год проводят специальные мероприятия по решению чеченского конфликта в чечне погиб командир


### Перплексия

Простыми словами - нам нужно расчитать вероятность текста (мы это уже научились делать выше) и возвести ее в степень (-1/N), где N это количество слов в тексте.

In [36]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [94]:
def compute_joint_proba(text, word_counts):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_counts:
            prob += (np.log(word_counts[word]/word_counts.total()))
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)


def compute_join_proba_markov_assumption_bi(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>'], 2):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)

def compute_join_proba_markov_assumption_tri(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>'], 3):
        word1, word2, word3 = ngram.split()
        bigram = ' '.join([word1, word2])
        unigram = word3
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)

Немного отложенных текстов

In [84]:
sentences_news2 = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] + ['<end>'] for text in sent_tokenize(news[-6000:])]
len(sentences_news2)

41

In [95]:
ps = []
for sent in sentences_news2:
    prob, N = compute_joint_proba(' '.join(sent), unigrams_news)
    ps.append(perplexity(prob, N))
print('Униграммная модель', np.mean(ps))

Униграммная модель 24556.427269220414


In [96]:
ps = []
for sent in sentences_news2:
    prob, N = compute_join_proba_markov_assumption_bi(' '.join(sent), unigrams_news, bigrams_news)
    ps.append(perplexity(prob, N))
print('Биграммная модель', np.mean(ps))

Биграммная модель 14886.525176766476


In [97]:
ps = []
for sent in sentences_news2:
    prob, N = compute_join_proba_markov_assumption_tri(' '.join(sent), bigrams_news, trigrams_news)
    ps.append(perplexity(prob, N))
print('Триграммная модель', np.mean(ps))

Триграммная модель 32112.552946369142


Ожидалось, что перплексия биграмной будет меньше униграммной, а триграммная будет ещё меньше. Но с триграммной что-то не так пошло. Но в семинарской тетрадке тоже на корпусе неожиданно получилось.

Задание № 2

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

- Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Можно в тесте давать только известные слова, но так делать нехорошо.
Можно все неизвестные слова заменять на \<UNK>. У нас заранне может быть фиксированный словарь и то, что в train в него не входит станет словом \<UNK>. Или мы составляем словарь из train и то, что встречается меньше n раз, делаем словом \<UNK>. В итоге у него тоже будет частота.

- Что такое сглаживание (smoothing)?

Способ решить проблему нулевой вероятности ситуации. Если слово попалось в неизвестном контексте (в котором в train не встречалось), чтобы не ставить вероятность =0, можно сказать, что вероятность маленькая (но не 0), отняв немного вероятности у частотных ситуаций.